In [52]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [53]:
# Load data
data_path = "../../data"
X_train = pd.read_csv(os.path.join(data_path, "X_train_pca.csv")).values
y_train_org = pd.read_csv(os.path.join(data_path, "y_train_pca.csv"))
X_test = pd.read_csv(os.path.join(data_path, "X_test_pca.csv")).values
y_test_org = pd.read_csv(os.path.join(data_path, "y_test_pca.csv"))
traits = ['Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional Stability', 'Openness']

In [54]:
# Create results directory
results_path = "../../results"
specific_results_path = os.path.join("../../results", "cnn_classification")
os.makedirs(results_path, exist_ok=True)
os.makedirs(specific_results_path, exist_ok=True)

In [55]:
def calc_roc_auc(y_true, y_pred):
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    n_classes = np.unique(y_true)
    roc_auc_scores = []
    for label in n_classes:
        # Create binary labels for the current class vs. all other classes
        y_true_class = (y_true == label).astype(int)
        y_pred_class = (y_pred == label).astype(int)
        
        # Calculate ROC AUC for the current class
        roc_auc = roc_auc_score(y_true_class, y_pred_class)
        roc_auc_scores.append(roc_auc)
    return roc_auc_scores

In [56]:
for trait in traits:
    print(f"Processing {trait}")
    trait_bin = trait + "_bin"
    label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
    y_train = np.array([label_mapping[label] for label in y_train_org[trait_bin]])
    y_test = np.array([label_mapping[label] for label in y_test_org[trait_bin]])

    num_classes = len(np.unique(y_test)) 

    # Define the CNN model
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.Dropout(0.5),  # Add dropout to reduce overfitting
        keras.layers.Dense(64, activation='relu'),
        keras.layers.Dense(10, activation='softmax')  # Adjust the output layer units based on your problem
    ])

    # Compile the model
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',  # Use this loss for multiclass classification
                metrics=['accuracy'])

    # Display the model summary
    model.summary()

    # Train the model
    batch_size = 64
    epochs = 200

    history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

    # Evaluate the model on the test data
    test_loss, test_accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {test_accuracy}")
    break
    # Save the trained model to a file
    # model.save("ecg_classification_model.h5")

Processing Extraversion
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 128)               1152      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_37 (Dense)            (None, 64)                8256      
                                                                 
 dense_38 (Dense)            (None, 10)                650       
                                                                 
Total params: 10058 (39.29 KB)
Trainable params: 10058 (39.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Epoch 1/200
27/27 [==============================] - 0s 4ms/step - loss: 2.0961 - accuracy: 0.3918 - val_loss: 1.7507 - val_accuracy: 0.5241
Epoch 2/200
27/27 [==============================] - 0s 1ms/step - loss: 1.4524 - accuracy: 0.4976 - val_loss: 1.1517 - val_accuracy: 0.5241
Epoch 3/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0928 - accuracy: 0.4970 - val_loss: 1.0277 - val_accuracy: 0.5027
Epoch 4/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0262 - accuracy: 0.5059 - val_loss: 0.9927 - val_accuracy: 0.5401
Epoch 5/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0211 - accuracy: 0.5036 - val_loss: 0.9804 - val_accuracy: 0.5294
Epoch 6/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0062 - accuracy: 0.5143 - val_loss: 0.9764 - val_accuracy: 0.5401
Epoch 7/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0031 - accuracy: 0.5042 - val_loss: 0.9683 - val_accuracy: 0.5508
Epoch 8/200
2